# GPT Spotify Playlist Generator

**NB!** This tutorial requires the use of paid services. See the OpenAI pricing [here](https://openai.com/api/pricing/). At the time of writing, the price is $0.150 per 1M tokens (medium sized words) using the GPT-4o-mini model.

This tutorial demonstrates how to:
1. Create a Spotify development user
2. Create a ChatGPT development user
3. Get the respective REST API credentials
4. Use the respective REST API's
5. Fetch songs from a spotify playlist
6. Randomly select a song and add it to another playlist
7. Use ChatGPT to pick a recommendation based on the fetched songs
8. Programmatically open the recommended song in the browser
9. Host a login endpoint for Spotify to get the user's credentials
10. Add the recommendation to the user's playlist

In [1]:
import os
from dotenv import load_dotenv
import requests
import json
from base64 import b64encode
import random
import webbrowser
from urllib.parse import urlencode
from flask import Flask, request, redirect
import secrets

## Create the .env file
The credentials and url's required for this project are stored in a `.env` file. This file is not included in the repository for security reasons. Please make a copy of the `.env.example` file and rename it to `.env`. Fill in the required fields with the information that becomes available through this tutorial.

## Create Spotift API Key

To create an API key with Spotify, you can follow these steps:

1. Go to the Spotify Developer Dashboard: [Spotify Developer Dashboard](https://developer.spotify.com/dashboard/)
2. Log in with your Spotify account or create a new account if you don't have one.
3. Create a new app by clicking on the "Create an App" button.
4. Fill in the required information for your app, such as the app name and app description. NB! The callback URL **MUST** be `http://localhost:8080/callback` for this to work.
5. Once your app is created, you will get a Client ID and a Client Secret under the "Settings" button.
6. Paste the Client ID and Client Secret into the .env file.

## Get Playlist IDs

The project requires a source playlist where recommendations will be taken from, and a target playlist where the recommendations will be pushed and stored.

To get the playlist ID:

1. Open the Spotify application or website.
2. Navigate to the playlist you want to use.
3. Click on the three dots (...) next to the playlist name.
4. Select "Share" from the dropdown menu.
5. Click on "Copy Playlist Link" to copy the link to your clipboard.
6. Paste the link into the .env file.


For more detailed instructions, you can refer to the Spotify Developer documentation: [Spotify Developer Documentation](https://developer.spotify.com/documentation/)

## Spotify Code

In [2]:
def extract_playlist_id(playlist_url):
    playlist_id = playlist_url.split("playlist/")[-1]
    playlist_id = playlist_id.split("?")[0]
    return playlist_id

In [3]:
# Load environment variables from .env file
load_dotenv()

# Get the playlist URL and access token from environment variables
source_playlist_url = os.getenv("SOURCE_PLAYLIST_URL")
target_playlist_url = os.getenv("TARGET_PLAYLIST_URL")
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

# Extract the playlist ID from the source playlist URL
source_playlist_id = extract_playlist_id(source_playlist_url)

# Extract the playlist ID from the target playlist URL
target_playlist_id = extract_playlist_id(target_playlist_url)

In [4]:
def get_spotify_access_token(client_id, client_secret):
    
    # https://developer.spotify.com/documentation/web-api/concepts/authorization
    # https://developer.spotify.com/documentation/web-api/tutorials/client-credentials-flow
    
    url = 'https://accounts.spotify.com/api/token'
    headers = {
        'Authorization': 'Basic ' + b64encode(f"{client_id}:{client_secret}".encode()).decode()
    }
    data = {
        'grant_type': 'client_credentials'
    }
    
    response = requests.post(url, headers=headers, data=data)
    
    if response.status_code == 200:
        return response.json().get('access_token')
    else:
        response.raise_for_status()

In [5]:
access_token = get_spotify_access_token(client_id, client_secret)

In [6]:
def fetch_top_songs(playlist_id, access_token, limit):
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    
    # https://developer.spotify.com/documentation/web-api/concepts/access-token
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    params = {
        "limit": limit
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        top_songs = response.json()
        return top_songs
    else:
        print("Failed to fetch top songs. Error:", response.status_code)
        print(response.json())
        return None

In [7]:
limit = 10
top_songs = fetch_top_songs(source_playlist_id, access_token, limit)

In [8]:
def create_track_label(item):
    return item["track"]["name"] + " [" + item["track"]["artists"][0]["name"] + "]"

In [9]:
for item in top_songs["items"]:
    print(create_track_label(item))

Die With A Smile [Lady Gaga]
Taste [Sabrina Carpenter]
Sigg [Ballinciaga]
Så længe jeg er sexy. [Annika]
BIRDS OF A FEATHER [Billie Eilish]
Espresso [Sabrina Carpenter]
Please Please Please [Sabrina Carpenter]
A Bar Song (Tipsy) [Shaboozey]
I Had Some Help (Feat. Morgan Wallen) [Post Malone]
Utested [Ari Bajgora]


In [10]:
random_song = random.choice(top_songs['items'])
track_url = random_song['track']['external_urls']['spotify']
print(create_track_label(random_song))

BIRDS OF A FEATHER [Billie Eilish]


**NB!** Paying for Chatgpt Plus will **NOT** grant you the credits required to use with the REST API. API use requires a separate payment.

To create a ChatGPT account and get the API key, follow these steps:

1. Go to the OpenAI website: [OpenAI](https://www.openai.com/)
2. Click on the "Sign Up" button to create a new account or "Log In" if you already have an account.
3. Follow the prompts to complete the sign-up process.
4. Once logged in, navigate to the API section of the website: [OpenAI API](https://platform.openai.com/account/api-keys)
5. Click on "Create new secret key" to generate a new API key.
6. Copy the generated API key into the .env file.
7. Add a payment method and add credit to your account to use the API. Add and monitor credit: [Open API](https://platform.openai.com/settings/organization/billing/overview).

For more detailed instructions, you can refer to the OpenAI documentation: [OpenAI Documentation](https://beta.openai.com/docs/)

## ChatGPT Code

In [11]:
# Load environment variables from .env file
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

In [12]:
role_prompt = """
You are going to recommend good music. You will respond exclusively with this exact template:
{
    "name": "Song Name",
    "url": "Spotify Song URL"
}
Once the user prompts you with a list of songs, you should pick one and respond with the template.
"""

In [13]:
def create_song_list(songs):
    songs_with_urls = []
    for item in songs["items"]:
        song = {
            "name": item["track"]["name"],
            "url": item["track"]["external_urls"]["spotify"]
        }
        songs_with_urls.append(song)
    return songs_with_urls

In [14]:
songs_with_urls = create_song_list(top_songs)
songs_with_urls[:3] # See what the data looks like

[{'name': 'Die With A Smile',
  'url': 'https://open.spotify.com/track/2plbrEY59IikOBgBGLjaoe'},
 {'name': 'Taste',
  'url': 'https://open.spotify.com/track/5G2f63n7IPVPPjfNIGih7Q'},
 {'name': 'Sigg',
  'url': 'https://open.spotify.com/track/4u6STUR9fsBQozW1S40BTN'}]

In [15]:
# Create a JSON string from the list of songs
prompt = json.dumps(songs_with_urls)
print(prompt)

[{"name": "Die With A Smile", "url": "https://open.spotify.com/track/2plbrEY59IikOBgBGLjaoe"}, {"name": "Taste", "url": "https://open.spotify.com/track/5G2f63n7IPVPPjfNIGih7Q"}, {"name": "Sigg", "url": "https://open.spotify.com/track/4u6STUR9fsBQozW1S40BTN"}, {"name": "S\u00e5 l\u00e6nge jeg er sexy.", "url": "https://open.spotify.com/track/0fFMt9cc1EEQDIWLGDcqKd"}, {"name": "BIRDS OF A FEATHER", "url": "https://open.spotify.com/track/6dOtVTDdiauQNBQEDOtlAB"}, {"name": "Espresso", "url": "https://open.spotify.com/track/2qSkIjg1o9h3YT9RAgYN75"}, {"name": "Please Please Please", "url": "https://open.spotify.com/track/5N3hjp1WNayUPZrA8kJmJP"}, {"name": "A Bar Song (Tipsy)", "url": "https://open.spotify.com/track/5fZJQrFKWQLb7FpJXZ1g7K"}, {"name": "I Had Some Help (Feat. Morgan Wallen)", "url": "https://open.spotify.com/track/5IZXB5IKAD2qlvTPJYDCFB"}, {"name": "Utested", "url": "https://open.spotify.com/track/19188WQf2DrNMfI1XLDnYF"}]


In [16]:
def access_chat_gpt_api(role_prompt, prompt, api_key):
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    data = {
        "model": "gpt-4o-mini",
        "messages": [
            {
                "role": "system",
                "content": role_prompt
            },
            {
                "role": "user",
                "content": prompt
            }
        ]
    }
    
    response = requests.post(url, headers=headers, json=data)
    
    if response.status_code == 200:
        return response.json()
    else:
        print("Failed to access ChatGPT API. Error:", response.status_code)
        print(response.json())
        return None

In [17]:
chat_gpt_response = access_chat_gpt_api(role_prompt, prompt, openai_api_key)

In [18]:
import json

# Pretty print the JSON response.
# The JSON formatted response text is located in choices.message.content
print(json.dumps(chat_gpt_response, indent=4))

{
    "id": "chatcmpl-A3pEFGQNhuHas2fweHYUj4Z3WHd3s",
    "object": "chat.completion",
    "created": 1725475271,
    "model": "gpt-4o-mini-2024-07-18",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": "{\n    \"name\": \"Die With A Smile\",\n    \"url\": \"https://open.spotify.com/track/2plbrEY59IikOBgBGLjaoe\"\n}",
                "refusal": null
            },
            "logprobs": null,
            "finish_reason": "stop"
        }
    ],
    "usage": {
        "prompt_tokens": 454,
        "completion_tokens": 39,
        "total_tokens": 493
    },
    "system_fingerprint": "fp_f33667828e"
}


In [19]:
chat_gpt_content = chat_gpt_response['choices'][0]['message']['content']

In [20]:
# Parse the content as JSON into a dict
parsed_content = json.loads(chat_gpt_content)

# Pretty print the content of the response
print(json.dumps(parsed_content, indent=4))

{
    "name": "Die With A Smile",
    "url": "https://open.spotify.com/track/2plbrEY59IikOBgBGLjaoe"
}


In [21]:
# Extract the content url
content_url = parsed_content['url']
content_url

'https://open.spotify.com/track/2plbrEY59IikOBgBGLjaoe'

In [22]:
# Find the original track info based on the URL
def find_track_by_url(url, songs):
    for item in songs['items']:
        if item['track']['external_urls']['spotify'] == url:
            return item
    return None

In [23]:
track_info = find_track_by_url(content_url, top_songs)

In [24]:
print(track_info)

{'added_at': '2024-09-04T09:34:08Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/'}, 'href': 'https://api.spotify.com/v1/users/', 'id': '', 'type': 'user', 'uri': 'spotify:user:'}, 'is_local': False, 'primary_color': None, 'track': {'preview_url': None, 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'TZ', 'UG', 'AG', 'AM', 'BS', 'BB', 'BZ', 'BT', 'BW', 'BF', 'CV', 'CW', 'DM', 'FJ', 'GM', 'GE', 'GD', 'GW', 'GY', 'HT',

In [25]:
# Programaticlaly open the url in the browser
webbrowser.open(content_url)
print(f"Opening the recommended song {parsed_content['name']} in your browser...")

Opening the recommended song Die With A Smile in your browser...


## Spotify User Login

While the Spotify Developer Account credentials allow for read access to publicly visible spotify resources, it does not grant write access to a user's playlists. Think of it this way: If you turned this project into a website, each user logging into the website would be logging in using his/her own Spotify account. This would allow the website to access the user's playlists and add recommendations. The credentials from earlier are only used to acquire the user's credentials. The following section and code is based on the [Spotify Authorization Docs](https://developer.spotify.com/documentation/general/guides/authorization-guide/), specifically the [Authorization Code Flow](https://developer.spotify.com/documentation/web-api/tutorials/code-flow), and demonstrates how to send the user to the Spotify login page, and how to extract the resulting credentials.

### Create Login API Endpoint

In [26]:
# Load the required environment variables
load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
redirect_uri = os.getenv("REDIRECT_URI")

# Create the Flask app used for defining API endpoints
app = Flask(__name__)

### Create the login handler

This function will run when the user navigates to [http://localhost:8080/login](http://localhost:8080/login). It redirects the user to the Spotify login website, with an URL that contains information about the app, as well as what resources the app is requesting access to. This will make more sense when you start the server and open the website later.

In [27]:
@app.route('/login')
def login():
    
    # Generate random 16 character string
    # to keep track of the login attempt
    state = secrets.token_urlsafe(16)
    
    # What the app can do with the user's Spotify account
    # As a rule, in case you get hacked, only ask for what you need
    # https://developer.spotify.com/documentation/web-api/concepts/scopes
    scope = " ".join(
        [
            # Images
            # "ugc-image-upload",
            # Spotify Connect
            "user-read-playback-state",
            "user-modify-playback-state",
            "user-read-currently-playing",
            # Playback
            "app-remote-control",
            "streaming",
            # Playlists
            "playlist-read-private",
            "playlist-read-collaborative",
            "playlist-modify-private",
            "playlist-modify-public",
            # Follow
            # "user-follow-modify",
            # "user-follow-read",
            # Listening History
            "user-read-playback-position",
            "user-top-read",
            "user-read-recently-played",
            # Library
            "user-library-modify",
            "user-library-read",
            # Users
            # "user-read-email",
            # "user-read-private",
            # Open Access
            # "user-soa-link",
            # "user-soa-unlink",
            # "soa-manage-entitlements",
            # "soa-manage-partner",
            # "soa-create-partner",
        ]
    )
    
    # Create the spotify login URL
    spotify_login_url = 'https://accounts.spotify.com/authorize?' + urlencode({
        # What type of Spotify login we want to perform
        'response_type': 'code',
        # Identifies this app (you got this from the spotify developer portal)
        'client_id': client_id,
        # Specifies what we want access to
        'scope': scope,
        # Specifies where spotify should send the user once the login process is complete
        'redirect_uri': redirect_uri,
        # A random unique string that lets us keep track of the login attempt.
        # It will be included in the url when spotify redirects the user back to us.
        'state': state
    })
    
    
    print("--------------------------------------------------------")
    print("User accessed the login endpoint, and is being redirected to Spotify Login")
    print("State:", state)
    print("Redirect Uri:", redirect_uri)
    print("Scope:", scope)
    print("Spotify Login URL (where the user just got redirected to):")
    print(spotify_login_url)
    print("--------------------------------------------------------")
    
    # Send the user to the Spotify login page
    return redirect(spotify_login_url)


### Create the callback handler

Once the user has logged in, the spotify login website will redirect the user back to the redirect_uri that was specified in the spotify_login_url above. In this case, the redirect_uri is [http://localhost:8080/callback](http://localhost:8080/callback). The function below will run when the user is redirected there. In addition to the redirect URL, Spotify will add URL parameters that can be used to fetch the user's credentials. The url should look something like this:
http://localhost:8080/callback?code=an_authorization_code_from_spotify&state=the_random_string_we_generated_earlier

In [28]:
@app.route('/callback')
def callback():
    global auth_options
    code = request.args.get('code', None)
    state = request.args.get('state', None)
    
    print("--------------------------------------------------------")
    print("Spotify redirected user back to us after completed login")
    print("Code:", code)
    print("State:", state)
    print("--------------------------------------------------------")

    if state is None:
        print("--------------------------------------------------------")
        print("State is missing. This request might not have been sent from Spotify.")
        print("--------------------------------------------------------")
        
        # Redirect the user to an empty URL with an error message
        return redirect('/#' + urlencode({'error': 'state_mismatch'}))
    else:
        # The user has been redirected back to us from Spotify login with a valid state.
        # Now we can use the code in the URL to get the user's access token.
        
        # The URL to spotify's token endpoint
        spotify_token_url = 'https://accounts.spotify.com/api/token'
        
        # The data required by the Spotify endpoint to get the access token
        request_body = {
            'code': code,
            'redirect_uri': redirect_uri,
            'grant_type': 'authorization_code'
        }
        
        # The headers required by the Spotify endpoint to get the access token
        headers = {
            'content-type': 'application/x-www-form-urlencoded',
            # The credentials of this app (same as was used in the function get_spotify_access_token())
            # This is included because when the user was redirected to spotify, the url contained the client_id
            # of this app, meaning the code the user got for logging in is only available to this app.
            'Authorization': 'Basic ' + b64encode(f"{client_id}:{client_secret}".encode()).decode()
        }
        
        # Send a POST request to the endpoint to exchange the code for an access token.
        response = requests.post(spotify_token_url, data=request_body, headers=headers)
        
        if response.status_code == 200:
            # If the request was successful, parse the response.
            token_data = response.json()
            
            # Write token data to a JSON file
            token_data_file = 'spotify_token_data.json'
            with open(token_data_file, 'w') as f:
                json.dump(token_data, f, indent=4)
            
            # Also show the access token to the user            
            return ({
                "message": "Successfully fetched access token",
                "access_token": token_data
            }, 200)
        else:
            # If the request was not successful, print an error message.
            print("--------------------------------------------------------")
            print("Failed to fetch access token. Error:", response.status_code)
            print(response.json())
            print("--------------------------------------------------------")
            # Send the error message to the user
            return ({"error": response.reason}, response.status_code)

### Host the login API

Read all the steps before continuing:

1. Start the login API using the cell below (do not click the URL's that will appear in the output. They don't work.)
1. Open the login url [http://localhost:8080/login](http://localhost:8080/login) in your browser
1. Look at the log messages in the output of the API notebook cell 
1. Log in with your Spotify account
1. You will be redirected to the callback URL
2. Look at the URl in the browser. Does anything look like the URL in the login and callback handler?
3. Look at the log messages in the output of the API notebook cell 
4. The credentials will be printed in the terminal, and saved to the [spotify_token_data.json](spotify_token_data.json) file.
5. When done, shut down the login API by clicking the stop button in the Jupyter notebook cell.

In [29]:
# Start the Flash server (login API)
app.run(port=8080, host="0.0.0.0")

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.1.23:8080
Press CTRL+C to quit


### Stop the login API

Remember to click the stop button in the Jupyter notebook cell to stop the login API, other wise you will not be able to run the rest of the code.

### Load the credentials from the JSON file

When the user logged in, the access token was saved to a JSON File [spotify_token_data.json](spotify_token_data.json) to make these demos easier to work with. Lets load them into the notebook.

In [30]:
# Load the token data from the JSON file
token_data_file = 'spotify_token_data.json'
with open(token_data_file, 'r') as f:
    token_data = json.load(f)

Extract the access token from the token_data.

In [31]:
user_access_token = token_data['access_token']

## Add the recommendation to the user's playlist

In [32]:
# Function to add a song to a Spotify playlist
# https://developer.spotify.com/documentation/web-api/reference/add-tracks-to-playlist
def add_song_to_playlist(song, playlist_id, user_access_token):
    # Add a song to a Spotify playlist
    
    response = requests.post(
        f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks",
        headers={
            "Authorization": f"Bearer {user_access_token}",
            "Content-Type": "application/json",
        },
        json={
            "uris": [song['track']['uri']],
            "position": 0,
        },
    )
    
    # Throw an error if the request was not successful
    response.raise_for_status()

In [33]:
# Add the random song to the target playlist
add_song_to_playlist(random_song, target_playlist_id, user_access_token)

print(f'Added the song "{create_track_label(random_song)}" to the target playlist!')

Added the song "BIRDS OF A FEATHER [Billie Eilish]" to the target playlist!


Now lets open the playlist in the browser and check that the song was added.

In [34]:
# Programaticlaly open the url in the browser
webbrowser.open(target_playlist_url)
print(f'Opening the recommended song "{create_track_label(random_song)}" in your browser...')

Opening the recommended song "BIRDS OF A FEATHER [Billie Eilish]" in your browser...
